<a href="https://colab.research.google.com/github/CleanPegasus/Aging-c-GAN/blob/master/Aging_cGAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
mkdir data

In [2]:
cd data

/content/data


In [3]:
!wget https://data.vision.ee.ethz.ch/cvl/rrothe/imdb-wiki/static/wiki_crop.tar

--2019-05-24 08:39:16--  https://data.vision.ee.ethz.ch/cvl/rrothe/imdb-wiki/static/wiki_crop.tar
Resolving data.vision.ee.ethz.ch (data.vision.ee.ethz.ch)... 129.132.52.162
Connecting to data.vision.ee.ethz.ch (data.vision.ee.ethz.ch)|129.132.52.162|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 811315200 (774M) [application/x-tar]
Saving to: ‘wiki_crop.tar’

wiki_crop.tar       100%[===================>] 773.73M  1.22MB/s    in 11m 45s 

2019-05-24 08:51:02 (1.10 MB/s) - ‘wiki_crop.tar’ saved [811315200/811315200]



In [0]:
!tar -xvf wiki_crop.tar

In [0]:
import math
import os
import time
from datetime import datetime
from scipy.io import loadmat

In [0]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from keras import Input, Model
from keras.applications import InceptionResNetV2
from keras.callbacks import TensorBoard
from keras.layers import Conv2D, Flatten, Dense, BatchNormalization, Reshape, concatenate, LeakyReLU, Lambda, K, Conv2DTranspose, Activation, UpSampling2D, Dropout
from keras.optimizers import Adam
from keras.utils import to_categorical
from keras_preprocessing import image

In [0]:
def load_data(wiki_dir, dataset= 'wiki'):
  
  meta = loadmat(os.path.join(wiki_dir, "{}.mat".format(datset)))
  
  full_path = meta[dataset][0, 0]["full_path"][0]
  
  dob = meta[dataset][0, 0]["dob"][0]
  
  photo_taken = meta[dataset][0, 0]["photo_taken"][0]
  
  age = [calculate_age(photo_taken[i], dob[i]) for i in range(len(dob))]
  
  images = []
  age_list = []
  
  for index, image_path in enumerate(full_path):
    
    images.append(image_path[0])
    age_list.append(age[index])
    
  return images, age_list

In [0]:
def calculate_age(taken, dob):
  birth = datetime.fromordinal(max(int(dob) - 366, 1))
  
  if birth.month < 7:
    return taken - birth.year
  else:
    return taken - birth.year - 1

In [0]:
def build_encoder():
  
  input_layer = Input(shape = (64, 64, 3))
  
  enc = Conv2D(filters = 32, kernel_size = 5, strides = 2, padding = 'same')(input_layer)
  enc = LeakyReLU(alpha = 0.2)(enc)
  
  enc = Conv2D(filters = 64, kernel_size = 5, strides = 2, padding = 'same')(enc)
  enc = BatchNormalization()(enc)
  enc = LeakyReLU(alpha = 0.2)(enc)
  
  enc = Conv2D(filters = 128, kernel_size = 5, strides = 2, padding = 'same')(enc)
  enc = BatchNormalization()(enc)
  enc = LeakyReLU(alpha = 0.2)(enc)
  
  enc = Conv2D(filters = 256, kernel_size = 5, strides = 2, padding = 'same')(enc)
  enc = BatchNormalization()(enc)
  enc = LeakyReLU(alpha = 0.2)(enc)
  
  enc = Flatten()(enc)
  
  enc = Dense(4096)(enc)
  enc = BatchNormalization()(enc)
  enc = LeakyReLU(alpha = 0.2)(enc)
  
  enc = Dense(100)(enc)
  
  model = Model(inputs = [input_layer], outputs = [enc])
  
  return model

In [0]:
def build_gen():
  
  latent_dim = 100
  num_classes = 6
  
  input_z_noise = Input(shape = (latent_dims,))
  input_label = Input(shape = (num_classes,))
  
  x = concatenate([input_z_noise, input_label])
  
  x = Dense(2048, input_dim = latent_dims + num_classes)(x)
  x = LeakyReLU(alpha = 0.2)(x)
  x = Dropout(0.2)(x)
  
  x = Dense(256 * 8 * 8)(x)
  x = BatchNormalization()(x)
  x = LeakyReLU(alpha = 0.2)(x)
  x = Dropout(0.2)(x)
  
  x = Reshape(8, 8, 256)(x)
  
  x = UpSampling2D(size = (2, 2))(x)
  x = Conv2D(filters = 128, kernel_size = 5, padding = 'same')(x)
  x = BatchNormalization(momentum = 0.8)(x)
  x = LeakyReLU(alpha = 0.2)(x)
  
  x = UpSampling2D(size = (2, 2))(x)
  x = Conv2D(filters = 64, kernel_size = 5, padding = 'same')(x)
  x = BatchNormalization(momentum = 0.8)(x)
  x = LeakyReLU(alpha = 0.2)(x)
  
  x = UpSampling2D(size = (2, 2))(x)
  x = Conv2D(filters = 3, kernel_size = 5, padding = 'same')(x)
  x = BatchNormalization(momentum = 0.8)(x)
  x = Activation('tanh')(x)
  
  model = Model(inputs = [input_z_noise, input_labels], outputs = [x])
  
  return model

In [0]:
def expand_label_input():
  x = K.expand_dims(x, axis = 1)
  x = K.title(x, [1, 32, 32, 1])
  return x

In [0]:
def build_disc():
  
  input_shape = (64, 64, 3)
  label_shape = (6,)
  
  image_input = Input(shape = input_shape)
  label_input = Input(shape = label_input)
  
  x = Conv2D(filters = 64, kernel_size = 3, strides = 2, padding = 'same')(image_input)
  x = LeakyReLU(alpha = 0.2)(x)
  
  label_input1 = Lambda(expand_label_input)(label_input)
  
  x = concatenate([x, label_input1], axis = 3)
  
  x = Conv2D(128, kernel_size = 3, strides = 2, padding = 'same')(x)
  x = BatchNormalization()(x)
  x = LeakyReLU(alpha = 0.2)(x)
  
  x = Conv2D(256, kernel_size = 3, strides = 2, padding = 'same')(x)
  x = BatchNormalization()(x)
  x = LeakyReLU(alpha = 0.2)(x)
  
  x = Conv2D(512, kernel_size = 3, strides = 2, padding = 'same')(x)
  x = BatchNormalization()(x)
  x = LeakyReLU(alpha = 0.2)(x)
  
  x = Flatten()(x)
  x = Dense(1, activation = 'sigmoid')(x)
  
  model = Model(inputs = [image_input, label_input], outputs = [x])
  
  return model